huggingface에서 skt/kogpt2-base-v2 모델을 가져와서 KMWP 데이터를 학습시키고 코드를 생성한 과정이 담겨 있습니다.

# 필요 라이브러리 설치

In [ ]:
# 트랜스포머 설치
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.6 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# 데이터셋 설치 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 53.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.4 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 63.8 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
     |████████████████████████████████| 144 kB 61.5 MB/s 
     |████████████████████████████████| 271 kB 68.4 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 112 kB 66.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
  Attempting unin

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf 
import re 

# 1) 데이터셋 가져오기

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#train_data_path = '/content/drive/MyDrive/Ko-Co/KMWP/data/train.csv'
train_data_path = '/content/drive/MyDrive/KMWP/code/data/new_train.csv'

data = pd.read_csv(train_data_path)
data.head()

,index,class,problem,code,answer,Unnamed: 5
0,0,8,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y),18,NaN
1,1,1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,a = 8\nb = 16\ny = b // a\nprint(y),2,NaN
2,2,8,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",a = 12\nb = 8\ny = a * b\nprint(y),96,NaN
3,3,3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",from itertools import permutations\n\na = ['0'...,987210,NaN
4,4,1,0.26 x 0.8을 계산해 주세요.,"a = 0.26\nb = 0.8\ny = a * b\nprint(""{:.2f}"".f...",0.21,NaN


> csv 파일을 dataset으로 만들어 처리합니다.

In [ ]:
from datasets import load_dataset 

dataset = load_dataset('csv', data_files= train_data_path,
                       split = 'train')

Using custom data configuration default-4978ea3626d3d918


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4978ea3626d3d918/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


> 따로 검증한 데이터를 사용했기 때문에 필요없는 컬럼은 지워줍니다.

In [ ]:
dataset = dataset.remove_columns(['index','Unnamed: 5'])
dataset

Dataset({
    features: ['class', 'problem', 'code', 'answer'],
    num_rows: 2820
})

# 2) GPT2 Finetune 하기
- 여기서는 검증데이터를 따로 분리하지 않고 데이터 전체를 학습에 이용했습니다.

In [ ]:
from transformers import AutoTokenizer 
from transformers import GPT2LMHeadModel 

model_path = 'skt/kogpt2-base-v2'

# 토크나이저 및 모델 가져오기
tokenizer = AutoTokenizer.from_pretrained(model_path, 
                                        bos_token = '</s>',
                                        sep_token='<sep>',
                                        eos_token='</s>',
                                        pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained(model_path)

def prepare_train_features(examples): 
  for i, j in enumerate(examples['problem']):
    examples['problem'][i] = j + '<sys>' + str(examples['class'][i]) + ' <sys>'

  tokenized_examples = tokenizer(
      text = examples['problem'],
      text_pair = examples['code'],
      padding = 'max_length',
      max_length = 260, 
      truncation = True
  )
  tokenized_examples['labels'] = tokenized_examples['input_ids'].copy()
  return tokenized_examples 


# 토큰화 된 데이터셋 생성 

#dictdataset = dataset.train_test_split(0.01) # 검증데이터 사용 시 쓰면 됨. 아래 dataset->dictdataset으로 바꿔주기
tokenized_datasets = dataset.map(prepare_train_features, batched=True,
                                     remove_columns = dataset.column_names)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

Parameter 'function'=<function prepare_train_features at 0x7f9990aa60e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3 [00:00<?, ?ba/s]

In [2]:
#tokenized_datasets['input_ids'][0]

In [ ]:
import numpy as np
from transformers import DataCollatorWithPadding 
from datasets import load_metric 

_collator = DataCollatorWithPadding(tokenizer = tokenizer) 
_metric = load_metric('accuracy') 

In [ ]:
def compute_metrics(p):
  preds, labels = p
  output = _metric.compute(predictions = np.argmax(preds, axis=-1), references=labels)
  return output

# 3) 모델 학습시키기

In [ ]:
from transformers import TrainingArguments, Trainer 

args = TrainingArguments('test_trainer')

In [ ]:
trainer = Trainer(
    model = model, 
    args = args, 
    data_collator=_collator,
    train_dataset = tokenized_datasets,#['train'],
    #eval_dataset = tokenized_datasets['test'],  # 검증데이터 사용 할 때 위 주석 다 풀어주면 됨
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2820
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1059


Step,Training Loss
500,0.440800
1000,0.227500


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1059, training_loss=0.32694565067885617, metrics={'train_runtime': 356.6953, 'train_samples_per_second': 23.718, 'train_steps_per_second': 2.969, 'total_flos': 1122535065600000.0, 'train_loss': 0.32694565067885617, 'epoch': 3.0})

> 학습과정에서 체크포인트가 저장됩니다.

In [ ]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2820
})

In [ ]:
# 토큰화 된 데이터셋 input_ids decode하여 확인하기
#tokenizer.decode(tokenized_datasets['train']['input_ids'][1]) # 검증 데이터 사용 시 확인용
tokenizer.decode(tokenized_datasets['input_ids'][10])

'진호가 가진 줄자의 길이는 6m 15 cm 이고 두은이가 가진 줄자의 길이는 0.25 m입니다. 진호와 두은이가 가진  줄자의 길이 합은 몇 cm인지 구해 보세요.<sys> 1 <sys> a = (6 * 100) + 15\nb = (0.25 * 100)\ny = int(a + b)\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [1]:
# 토큰화 된 데이터셋 attention_mask 확인
#tokenized_datasets['attention_mask']

# 4) 모델 저장하기
- 모델과 토크나이저 모두 저장해야 합니다

In [ ]:
trainer.save_model('my_kogpt2_model_new') 
tokenizer.save_pretrained('my_kogpt2_model_new')

Saving model checkpoint to my_kogpt2_model_new
Configuration saved in my_kogpt2_model_new/config.json
Model weights saved in my_kogpt2_model_new/pytorch_model.bin
tokenizer config file saved in my_kogpt2_model_new/tokenizer_config.json
Special tokens file saved in my_kogpt2_model_new/special_tokens_map.json
tokenizer config file saved in my_kogpt2_model_new/tokenizer_config.json
Special tokens file saved in my_kogpt2_model_new/special_tokens_map.json


('my_kogpt2_model_new/tokenizer_config.json',
 'my_kogpt2_model_new/special_tokens_map.json',
 'my_kogpt2_model_new/vocab.json',
 'my_kogpt2_model_new/merges.txt',
 'my_kogpt2_model_new/added_tokens.json',
 'my_kogpt2_model_new/tokenizer.json')

# 5) 코드 생성하기 
generate() 함수를 사용하여 코드가 어떻게 생성되는지 살펴봅니다.

In [ ]:
#input_text = '둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 때, 정육각형의 한 변의 길이는 몇 cm인지 구해보세요.'
input_text = test_data.problem[8]
device = 'cuda:0'
text_ids = tokenizer.encode(input_text, return_tensors='pt') 
text_ids = text_ids.to(device) 
model = model.to(device)

a = model.generate(text_ids,
                   max_length = 260)
                   

In [ ]:
a

tensor([[ 9120, 25308,  9088, 22633,  9239, 46459,  9056,  8148, 10181, 22179,
         16691,  9120, 16504, 16576,  9239, 12789,  7847,  7958, 27266,  8149,
         11097, 16386,  8306,  8137, 11088,  8017, 16691,  9088, 22633, 16576,
          9239, 12789,  7847,  7958, 17347, 11097, 14476,  8306,  8137, 11088,
          8017, 16691, 12557, 11097,  9056,  8148, 10181, 44434, 12789, 10583,
         22031,  9267, 31954,  6969,  8084,   406,     4,  9247,   739,     4,
         13612, 18124,  9545,   375,   440, 18124,  9247,   375,   441, 18124,
         14476,   375,   442, 18124,  9143,   375,   443, 18124, 14476,   375,
           444, 18124, 11925,  9571, 10477,  9571,  9440,   439, 10340, 13726,
         10932,   442, 10227, 24646, 18124, 23622,   382,  6835,  7584,   382,
           401, 13612, 10340, 13726,   387,  9194,  7801,  8806,   382,   401,
         16901, 10340, 13063,   467,   375,   463, 18124, 11849, 29942, 29937,
         10227, 24646,   387, 22272,   443,   463,  

In [ ]:
tokenizer.decode(a[0])#, skip_special_tokens=True)

"강미와 세희는 같은 세종대왕 위인전을 샀습니다. 강미는 매일 같은 쪽수씩 일주일동안 138쪽을 읽었습니다. 세희는 매일 같은 쪽수씩 6일동안 175쪽을 읽었습니다. 하루동안 위인전을 읽은 쪽수는 누가 더 많을까요?<sys> 7 <sys> a = 13\nb = 7\nc = 175\nd = 5\ne = 175\nf = c - d - (a * b)\ndicts = {'강미': a * b, '세희': e * f}\ny = max(dicts, key=dicts.get)\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 쌓은 작은 수에서 작은 수에서 작은 수를 뺀 값을 뺀 값을 소수라 한다면, '세희': e * f}\ny = min(dicts, key=dicts.get)\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 쌓은 작은 수를 소수라면,"

생성된 코드를 decode 해보니, 

problem + class + code 가 모두 나타납니다. 

(즉, 문제에 대한 class와 code 모두를 예측 및 생성하고 있습니다)

\<pad\> 이후에도 언어가 생성되는데, 무시하겠습니다. 

# 6) 생성된 코드 처리
- test.csv에 담긴 problem에 대한 코드들을 생성하고
- 함수로 \<pad> 이후의 언어들을 무시합니다.
- 코드에 대한 답을 계산하여 저장합니다.


In [ ]:
# 데이터 불러오기
test_data_path = '/content/drive/MyDrive/Ko-Co/KMWP/data/test.csv'

test_data = pd.read_csv(test_data_path)
test_data.head() 

,idx,problem
0,1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...
1,2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...
2,3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...
3,4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?"
4,5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ..."


In [ ]:
test_data.problem[0]

'둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 때, 정육각형의 한 변의 길이는 몇 cm인지 구해보세요.'

In [ ]:
# 코드를 생성하여 리스트로 반환하는 함수
def result_show(test_data, model, tokenizer, device = 'cuda:0'):
  decoded_code_list = []
  for i in range(len(test_data)):
    input_text = test_data.problem[i]
    text_ids = tokenizer.encode(input_text, return_tensors='pt')
    text_ids = text_ids.to(device)
    model = model.to(device) 

    generated_code_id = model.generate(text_ids,
                                  max_length = 256)
    decoded_code = tokenizer.decode(generated_code_id[0])

    decoded_code_list.append(decoded_code)

  return decoded_code_list

In [ ]:
len(test_data)

282

In [ ]:
# 코드 생성
code_list = result_show(test_data, model, tokenizer, device='cuda:0')
#code_list

In [ ]:
code_list[5]

'무게가 같은 호박 2개의 무게는 200g이고 당근 한 개의 무게는 65g이며 무게가 같은 오이 4개의 무게는 164g입니다. 한 개의 무게가 가장 가벼운 것은 무엇인가요?<sys> 7 <sys> a = 2\nb = 2\nc = 165\nd = 2\ne = 165\ny = (a * b) // c\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 붙은      if c == (c // 2 - b) // d\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 붙은       y = d // 2\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad> 붙은     y = b // 2\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad> 붙은   y = d // 2\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 붙은 y - min(c)\nprint(y)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 붙은 y - min'

> 위 결과에서 \<sys> 와 \<pad> 사이의 코드만을 뽑아 저장하려고 합니다.

In [ ]:
# 하나의 추론 결과(str)에서 파이썬 코드(str) 추출
def get_code(item: str) -> str:
    return item[item.find('<sys>') + 13:item.find('<pad>')].strip()


# 결과들의 이터러블에 대한 파이썬 코드들 추출
def get_codes(item_list: iter) -> map:
    return list(map(get_code, item_list))
    # 상황이나 필요에 따라 list(map(get_code, item_list)) 등으로 수정


# 하나의 코드에 대해 성공하면 print 값을, 실패하면 해당 에러 메시지 반환
# print 문은 실제로 실행하지 않기 때문에 콘솔 출력이 없음
def get_answer(code: str) -> str:
    try:
        body, answer = code.split('print')
    except ValueError as e:
        return 'print 문이 없습니다.' if str(e).startswith('n') \
            else 'print 문이 여러 개입니다.'
        # if str(e) == 'not enough values to unpack (expected 2, got 1)'
    try:
        exec(body)
        return str(eval(answer[1:-1]))
    except Exception as e:
        return str(e)


# 코드의 이터러블에 대한 get_answer 값들을 반환
def get_answers(codes: iter) -> map:
    return list(map(get_answer, codes))
    # 상황이나 필요에 따라 list(map(get_answer, codes)) 등으로 수정

In [ ]:
# 코드만 잘 나오는지 확인
get_code(code_list[5])

'a = 2\nb = 2\nc = 165\nd = 2\ne = 165\ny = (a * b) // c\nprint(y)'

In [ ]:
# 코드들만 추출하여 리스트로 저장
answer_codes = get_codes(code_list)

In [ ]:
answer_codes[:2]

['a = 4\nb = 12\nc = 12\ny = c * b // a\nprint(y)',
 'a = 60\nb = 120\nc = a - b\ny = c - a\nprint(y)']

In [ ]:
len(answer_codes)

282

In [ ]:
# 코드에 대한 답이 잘 나오는지 확인
get_answer(answer_codes[1])

'-120'

In [ ]:
# 코드에 대한 답들을 리스트에 저장
answers = get_answers(answer_codes)

In [ ]:
answers[:2]

['36', '-120']

# 7) json 파일로 변환하기

In [ ]:
submit_data = test_data.copy() 

submit_data.head()

,idx,problem
0,1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...
1,2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...
2,3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...
3,4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?"
4,5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ..."


In [ ]:
submit_data['code'] = answer_codes
submit_data['answer'] = answers 

submit_data.head()

,idx,problem,code,answer
0,1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...,a = 4\nb = 12\nc = 12\ny = c * b // a\nprint(y),36
1,2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...,a = 60\nb = 120\nc = a - b\ny = c - a\nprint(y),-120
2,3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...,a = 8\nb = 29\ny = a * b\nprint(y),232
3,4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?",a = 20\nb = 12000\ny = a * b\nprint(y),240000
4,5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ...","from itertools import permutations\n\na = [8, ...",98652


In [ ]:
submit_data.set_index('idx', inplace=True) 
submit_data.head()

,problem,code,answer
idx,,,
1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...,a = 4\nb = 12\nc = 12\ny = c * b // a\nprint(y),36
2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...,a = 60\nb = 120\nc = a - b\ny = c - a\nprint(y),-120
3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...,a = 8\nb = 29\ny = a * b\nprint(y),232
4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?",a = 20\nb = 12000\ny = a * b\nprint(y),240000
5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ...","from itertools import permutations\n\na = [8, ...",98652


In [ ]:
submit_data_path = '/content/drive/MyDrive/Ko-Co/KMWP/data/KoGPT_v3_new.json'
submit_data.to_json(submit_data_path, orient='index', indent = 4)